In [ ]:
%mavenRepo s3 https://djl-ai.s3.amazonaws.com/dev

In [ ]:
%maven ai.djl:api:0.1.0
%maven ai.djl:repository:0.1.0
%maven ai.djl.mxnet:mxnet-engine:0.1.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.1.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
// %maven ai.djl.mxnet:mxnet-native-mkl:jar:osx-x86_64:1.6.0

In [ ]:
%%loadFromPOM
  <repositories>
    <repository>
      <id>djl.ai</id>
      <url>https://djl-ai.s3.amazonaws.com/dev</url>
    </repository>
  </repositories>

  <dependencies>
    <dependency>
      <groupId>ai.djl.mxnet</groupId>
      <artifactId>mxnet-native-mkl</artifactId>
      <version>1.6.0</version>
      <classifier>osx-x86_64</classifier>
    </dependency>
  </dependencies>


In [ ]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import ai.djl.mxnet.zoo.*;
import ai.djl.*;
import ai.djl.inference.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.index.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.*;
import ai.djl.util.*;


In [ ]:
var imageFile = Paths.get("../examples/src/test/resources/3dogs.jpg").toRealPath();
var img = BufferedImageUtils.fromFile(imageFile);
img

In [ ]:
var criteria = new HashMap<String, String>();
criteria.put("size", "512");
criteria.put("backbone", "resnet50");
criteria.put("dataset", "voc");
var model = MxModelZoo.SSD.loadModel(criteria);

In [ ]:
var predictor = model.newPredictor();
var detections = predictor.predict(img);

In [ ]:
ImageVisualization.drawBoundingBoxes(img, detections);
img